In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

from datetime import datetime as dt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

import statsmodels.api as sm
import statsmodels.graphics as smg
import statsmodels.stats as sm_stats
import statsmodels.tsa.api as tsa

from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
from typing import List

# Commits Bug Classification and Analysis

**This project aims to leverage "Github Commit Nessages Dataset" to assess the quality of the software among 32 popular open source projects hosted in Github using their commits messages to track corrections in code which is a signal for bad quality in develpoment**

In [ ]:
df = pd.read_csv('/kaggle/input/github-commit-messages-dataset/full.csv', parse_dates=True, infer_datetime_format=True)

### Pre-processing

In [ ]:
df['date'] = df['date'].apply(lambda x: dt.strptime(x,"%a %b %d %H:%M:%S %Y %z"))
#df['date'].apply(lambda x: dt.strptime(x,"%z "))

In [ ]:
df['date'] = pd.to_datetime(df['date'], utc=True)

In [ ]:
df['dday'] = df['date'].dt.date
df['wday'] = df['date'].dt.weekday
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['hour'] = df['date'].dt.hour
df['min'] = df['date'].dt.minute
df['sec'] = df['date'].dt.second

In [ ]:
df = df.join(df['author'].str.split('(<)',n=2,expand=True ))

In [ ]:
df = df.drop('author', axis=1)
df = df.drop(1, axis=1)
df = df.rename({0:'user', 2:'email' }, axis=1)
df['email'] = df['email'].str.strip('>')

In [ ]:
df.head()

## Repos Description

### Languages

In [ ]:
"""
for repo in df['repo'].unique():
    print(repo)
    #print('https://api.github.com/repos/'+repo+'/languages')
    repo_lang = pd.read_json('https://api.github.com/repos/'+repo+'/languages', orient='index')
    repo_lang.rename({0:'language'})
    print(repo_lang)
    #repo_lang.plot.pie(y=0, figsize=(6,6)).set_title(repo)
"""

In [ ]:
df.head()

In [ ]:
#X =  vectorizer.fit_transform(df['message'])

In [ ]:
#X.toarray()

## Exploratory Data Analysis

In [ ]:
print(len(df['repo'].unique()))
df['repo'].unique()

In [ ]:
ustats = df.groupby('repo').nunique()

In [ ]:
ustats = ustats.sort_values(by="message", ascending=False)

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1, figsize=(18,8))
fig.autofmt_xdate(rotation=90)
ax1.bar(x=ustats.index, height=ustats['message'])
ax1.set_title('Count of Commits')
ax2.bar(x=ustats.index, height=ustats['user'])
ax2.set_title('Count of Users')

In [ ]:
ustats['message'].plot.bar()

In [ ]:
ustats['message'].plot.pie(figsize=(10,10), autopct='%1.1f%%',
        shadow=True, startangle=90)

In [ ]:
ustats['user'].plot.bar()

In [ ]:
ustats.plot.scatter(x='user',y='message')

In [ ]:
ustats.drop('torvalds/linux').plot.scatter(x='user',y='message') 

In [ ]:
ustats.drop('torvalds/linux').sort_values(by='user',ascending=False)['user'].plot.bar()

In [ ]:
ustats.drop('torvalds/linux').sort_values(by='message',ascending=False)['message'].plot.bar()

### Time series

In [ ]:
df_sort = df.sort_values('dday')

In [ ]:
df_sort.head(20)

In [ ]:
message_day = df.pivot_table(index='dday', columns='repo', values='message', aggfunc='count', fill_value=0, margins=True)

In [ ]:
message_day.drop('All')[100:-5].rolling(90)['All'].mean().plot(figsize=(20,6))

In [ ]:
message_day.drop('All')[100:-5].rolling(30)['All'].mean().plot(figsize=(20,6))

In [ ]:
message_day.columns

In [ ]:
message_day.drop('All')[100:-5].rolling(30)['torvalds/linux'].mean().plot(figsize=(20,6))

In [ ]:
message_day.drop('All')[100:-5].rolling(30)['freebsd/freebsd-src'].mean().plot(figsize=(20,6))

In [ ]:
message_day.drop('All')[8500:-5].rolling(30)['pytorch/pytorch'].mean().plot(figsize=(20,6))

In [ ]:
message_day.drop('All')[8500:-5].rolling(30)['tensorflow/tensorflow'].mean().plot(figsize=(20,6))

In [ ]:
message_day.drop('All')[6500:-5].rolling(30)['scikit-learn/scikit-learn'].mean().plot(figsize=(20,6))

In [ ]:
df_user = df[['dday','repo','user']].drop_duplicates()

In [ ]:
user_day = df_user.pivot_table(index='dday', columns='repo', values='user', aggfunc='count', fill_value=0, margins=True)

In [ ]:
user_day.drop('All')[8500:-5].rolling(30)['pytorch/pytorch'].mean().plot(figsize=(20,6))

In [ ]:
user_day.drop('All')[8500:-5].rolling(30)['tensorflow/tensorflow'].mean().plot(figsize=(20,6))

### Competitors

node x deno
tensorflow  x pytorch
linux x freebsd
iphython x rstudio
matplotlib x ggplot2
go x rust
llvm x gcc ?
httpd x nginx

In [ ]:
test = df.loc[1:100]
test

# Commit Classification 

Reference
* Idan Amit, and Dror G. Feitelson. (2020). The Corrective Commit Probability Code Quality Metric.
* https://arxiv.org/abs/2007.10912
* https://github.com/evidencebp/commit-classification


Language_utils

In [ ]:
regex_list : List[str]

SCHEMA_NAME = 'general'
file_scheme = '([a-zA-Z0-9_\*\.])+\.[a-zA-Z]{1,4}'

REGULAR_SUFFIX = '(?:s|ed|ing)?'
VERB_E_SUFFIX = '(?:e|es|ed|ing)'

NEAR_ENOUGH = '[\S\s]{0,40}'

#term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|\'|\"|/|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|-|\|)" # Adding - should be tuned
#term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|\'|\"|/|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|\|)"
#term_seperator = "[^abcdefghijklmnopqrstuvwxyz]"
term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|\'|\"|/|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|_|\|)"
#term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|/|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|_|\|)" # no ",'
#term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|\'|\"|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|_|\|)" # without /

# Negation
negation_terms = ["aren't",
 'arent',
 "can't",
 'cannot',
 'cant',
 'could not',
 "couldn't",
 'couldnt',
 "didn't",
 'didnt',
 "doesn't",
 'doesnt',
 "don't",
 'dont',
 "hasn't",
 "haven't",
 "isn't",
 'isnt',
 'lack',
 "n't",
 'never',
 'no',
 'nobody',
 'none',
 'not',
 'nothing',
 "shouldn't",
 'shouldnt',
 "weren't",
 'werent',
 'without',
 "won't",
 'wont',
 "wouldn't",
 'wouldnt']

# TODO - consider adding if, maybe
modals = [#'if', 'maybe',
          'can', 'could', 'ha(?:ve|s|d)', 'may', 'might', 'must', 'need', 'ought', 'shall'
    , 'should', 'will', 'would']



# TODO - check https://arxiv.org/pdf/2001.09148.pdf for more

security_terms = [
 'advisory',
 'attack',
 'authenticat(e|ion)',
 'brute force', # consider
 'bug bount(y|ies)',
 'bypass', # consider
 'constant time',
 'crack',
 'credential(s)?',
 'cross-origin',
 'cross site',
 'cve(-d+)?(-d+)?',
 'clickjack',
 'cyber',
 'denial of service',
 '(de)?serializ', # consider
 'directory traversal',
 'dos', # consider
 'exploit',
 'expos(e|ing)',
 'hack',
 'hijack',
 'harden',
 #'infinite loop', # consider
 'injection',
 '(in)?secur(e|ity)',
 'lockout',
 'malicious',
 'malware',
 'nvd' # NVD
 'open redirect',
 'osvdb', # OSVDB
 'overflow', # consider
 'password(s)?',
 'permission(s)?',
 'poison',
 'port scan',
 'privilege',
 # 'proof of concept', # consider
 'rce', # remote code execution
 'redos' # ReDoS
 'remote code execution',
 'return oriented programming',
 'security',
 'session fixation',
 'spoof',
 'threat',
 'timing', # consider
 'traversal',
 'unauthori[z|s]ed',
 'vulnerabilit(?:y|ies)',
 'x(?: |-)frame(?: |-)options',
 'xss',
 'xsrf', # XSRF
 'xxe' # XXE
    ]


documentation_entities = [
    'change(?:s)?(?: |-)?(list|log|set|file)',
    'comment(s)?',
    'copy(?: |-)?right(?:s)?',
    'doc(?:s)?',
    'documentation',
    'explanation(?:s)?',
    'man(?: |-)?page(?:s)?',
    'manual',
    'note(?:s)?',
    'readme(?:.md)?',
    r'[-a-z\d_/\\]*.(md|txt)',
    'translation(?:s)?',
    'java(?: |-)?doc(?:s)?',
    'java(?: |-)?documentation',
    'example(?:s)?',
    'diagram(?:s)?',
    'guide(?:s)?',
    'gitignore',
    'icon(?:s)?',
    'doc(?: |-)?string(?:s)?',
    'tutorials(?:s)?',
    'help',
    'man',
    'doc(?: |-)?string(?:s)?',
    'desc(?:ription)?(?:s)?',
    'copy(?: |-)?right(?:s)?',
    'explanation(?:s)?',
    'release notes',
    'tag(?:s)?', # Git commit tags

]

prefective_entities = documentation_entities +[
    'indentation(?:s)?'
    , 'style'
    , 'todo(s)?'
    , 'typo(s)?'
    , 'verbosity']

software_goals = ['abstraction', 'coherence', 'cohesion', 'complexity', 'correctness', 'coupling', 'dependability'
    , 'duplication', 'efficiency', 'extensibility', 'flexibility' ,'maintainability', 'naming', 'performance', 'portability', 'quality'
    , 'readability', 'reliability', 're(?:-| )?use' ,'re(?:-| )?usability', 'security', 'simplicity', 'testability', 'testable', 're(?:-| )?usable'
    , 'readable', 'portable', 'maintainable', 'flexible', 'efficient', 'encapsulation'
                  ]

software_goals_modification = [
    'better','improv(?:e|es|ed|ing)', 'increas(?:e|es|ed|ing)', 'reduc(?:e|es|ed|ing)', 'worse', 'make', 'more', 'less'
]

software_entities = ['algorithm(?:s)?',
 'class(?:es)?',
 'collection(?:s)?',
 'constant(?:s)?',
 'constructor(?:s)?',
 'field(?:s)?',
 'function(?:s)?',
 'interface(?:s)?',
 'member(?:s)?',
 'method(?:s)?',
 'module(?:s)?',
 'parameter(?:s)?',
 'procedure(?:s)?',
 'routine(?:s)?`',
 'structure(?:s)?',
 'template(?:s)?',
 'type(?:s)?',
 'unit(?:s)?',
]

software_terms = [ 'assertion(?:s)?', 'assignment(?:s)?',  'code',  'conditional(?:s)?',  'control', 'definition(?:s)?'
    , 'delegate', 'delegation'
    , 'design pattern(?:s)?', 'error(?:-| )?code(?:s)?', 'exception(?:s)?',  'flag(?:s)?',  'getter(?:s)?'
    , 'guard clause(?:s)?', 'hierarch(?:y|ies)', 'implementation(?:s)?', 'inheritance', 'inline'
    ,  'internal', 'macro(?:s)?'
    , 'magic number(?:s)?', 'modifier(?:s)?', 'null object(?:s)?', 'object(?:s)?'
    , 'patch(?:es)?',  'pointer(?:s)?', 'polymorphism', 'quer(?:y|ies)',  'reference(?:s)?'
    , 'ref(?:s)?'
    , 'return type', 'setter(?:s)?', 'static',  'sub(?:-| )?class(?:es)?', 'super(?:-| )?class(?:es)?'
    , '(?:sub)?(?:-| )?system(?:s)?'
    , 'uninline'
    #, 'value(?:s)?'
    , 'variable(?:s)?', 'handler', 'plugin'
    #, '(?:in)?validation'
    #, 'input', 'output'
    , 'contravariant', 'covariant'
                  # , 'link(?:s)?'
    ,
                  'action(?:s)?'
                  # , 'event(?:s)?'
    , 'queue(?:s)?', 'stack(?:s)?'
    #, 'change(?:\s)?log'
    , 'driver(?:s)?'
    #, 'hook(?:s)?'
    #, 'target(?:s)?'
    , 'storage', 'tool(?:s)?',  'log(?:s)?', 'setting(?:s)?'
    #, '(?:index|indexes|indices)'
    , 'fall(?: |-)back(?:s)?', 'memory', 'param(?:s)?', 'volatile', 'file(?:s)?'
    , 'generic(?:s)?'
    #, 'test(?:s)?'
    , 'initialization(?:s)?', 'public', 'protected', 'private' ,'framework', 'singelton', 'declaration(?:s)?'
    , 'init' , 'destructor(?:s)?', 'instances(?:s)?', 'primitive(?:s)?'
    #, 'middle man'
    #, 'hierarchy'
                  ] + software_entities


# Well, we need them...
unnedded_terms = ['unnecessary', 'unneeded', 'unused', '(?:not|never|no longer) used'
    #, 'old'
    , 'no longer needed', 'redundant', 'useless', 'duplicate(?:d)?', 'deprecated', 'obsolete(?:d)?', 'commented']


static_analyzers = ['lint', 'pylint', 'tslint', 'jlint', 'jslint', 'eslint', 'klint', 'xlint', 'linter']

code_review_fixes = ['(cr|pr)(s)?(-)?(d+)?\sfix(es)?', 'fix(?:ing|es|ed)?\s(cr|pr|code review|code-review|review)']

no_message = ['no message', 'wip', 'work in progress', 'message', 'change(?:-|\s)?set', 'commit']

programming_languges = [i.lower() for i in ['Python', 'JavaScript', 'Java', 'C\+\+', 'PHP', 'TypeScript', 'C',
       'C\#', 'Go', 'Ruby', 'HTML', 'Shell', 'CSS', 'Kotlin', 'Scala',
       'Swift', 'Jupyter Notebook', 'Rust', 'Perl', 'Lua', 'Haskell', 'R',
       'Objective\-C', 'Groovy', 'Vue', 'PowerShell', 'TSQL', 'Dart',
       'Clojure', 'MATLAB', 'Emacs Lisp', 'OCaml', 'Erlang', 'Elixir',
       'CoffeeScript', 'TeX', 'Fortran', 'Assembly', 'Vim script',
       'PLpgSQL', 'Makefile', 'Julia', 'BitBake', 'F\#', 'Common Lisp',
       'Vala', 'Coq', 'Smalltalk', 'Scheme', 'Visual Basic .NET',
       'Puppet', 'HCL', 'Smarty', 'Dockerfile', 'XSLT', 'GLSL', 'Haxe',
       'Cuda', 'Ada', 'SQF', 'Pascal', 'PLSQL', 'Gherkin', 'Jsonnet',
       'Nix', 'Roff', 'Apex', 'QML', 'CMake', 'D', 'Perl 6',
       'Visual Basic', 'Objective\-C\+\+', 'Prolog', 'Mathematica',
       'Batchfile', 'Reason', 'Markdown', 'DM', 'Elm', 'FreeMarker',
       'ABAP', 'M4', 'SystemVerilog', 'AutoHotkey', 'Verilog', 'IDL',
       'Tcl', 'Rich Text Format', 'SaltStack', 'UnrealScript', 'Zig',
       'WebAssembly', 'RAML', 'F\*', 'Stan', 'ColdFusion', 'Factor',
       'LLVM', 'Pike', 'VBA', 'Isabelle', 'OpenSCAD', 'ASP', 'Arc',
       'Racket', 'LookML', 'SMT', 'q', 'Xojo', 'ZenScript', 'Ceylon',
       'Agda', 'Limbo', 'SuperCollider', 'Pawn', 'xBase', 'JSON', 'Nim',
       'M', 'XC', 'SourcePawn', 'GDScript', 'LilyPond', 'SQLPL',
       'PostScript', "Ren'Py", 'Gnuplot', 'OpenEdge ABL',
       'Common Workflow Language', 'Xtend', 'Mercury', 'Genshi',
       'Open Policy Agent', 'RobotFramework']]


def build_sepereted_term(term_list : List, just_before =False):
    if just_before:
        sep = "%s(%s)" % (term_seperator, "|".join(term_list))
    else:
        sep = "%s(%s)%s" % (term_seperator, "|".join(term_list), term_seperator)
    return sep


def build_non_positive_linguistic(positive_re
                                  , neg=negation_terms):

    non_actionable_context = ['for(?:get|gets|got|getting)'
        , 'allow(s|ed|ing)?']


    return '(?:%s)' % "|".join([
        ('(?:%s)' + NEAR_ENOUGH + '(?:%s)') % (build_sepereted_term(modals, just_before=True)
                                      ,  positive_re)
        , ('(?:%s)' + NEAR_ENOUGH + '(?:%s)') % (build_sepereted_term(neg, just_before=True)
                                        ,  positive_re)
        , ('(?:%s)' + NEAR_ENOUGH + '(?:%s)') % (build_sepereted_term(non_actionable_context, just_before=True)
                                        ,  positive_re)
        # TODO - take care of documentation entities spereatly
        #, '(?:%s)[\s\S]{0,10}(?:%s)' % (build_sepereted_term(documentation_entities, just_before=True)
        #                                ,positive_re)
    ])


def match(commit_text, regex):
    text = commit_text.lower()

    return len(re.findall(regex, text))


def regex_to_big_query(reg_exp
                       , text_field='message'):
    # TODO - check
    # Take care of encoding
    reg_exp = reg_exp.replace("\\", "\\\\").replace("'", "\\'")
    #reg_exp = reg_exp.replace("\\\\", "\\")
    # No need for grouping
    reg_exp = reg_exp.replace("(?:", "(")
    str = "(" + "LENGTH(REGEXP_REPLACE(lower(" + text_field + ")," + "'%s', '@'))" % reg_exp + "-" \
          + "LENGTH(REGEXP_REPLACE(lower(" + text_field + ")," + "'%s', ''))" % reg_exp + ")"

    return str


def generate_bq_function(func_name
                         , code_generator
                         , commit: str ='XXX'):
    print("# Run in Standard sql ")
    print("CREATE OR REPLACE FUNCTION ")
    print(func_name)
    print(" (message string) ")
    print(" RETURNS int64 ")
    print("AS (")
    print("# Model language based on commit: {commit} ".format(commit=commit))
    code_generator()
    print(" ) ")
    print(" ; ")


def normalize(string):
    string = re.sub(r"\s+", " ", string.strip())
    while "  " in string:
        string = string.replace("  ", " ")
    return string

def print_logic_to_bq(regex_func
                      , concept):
    print("# " + concept)
    print( regex_to_big_query(regex_func()))
    print("# " + concept + " - end")

Conventional Commits

In [ ]:
cc_adaptive_terms = ['feat' # Feature
                        , 'build'
                        , 'chore'
                        , 'ci' # continuous integration
                        , 'test'
                        , 'perf' # performance
                     ]
cc_corrective_terms = ['fix']
cc_perfective_terms = ['docs', 'style']
cc_refactor_terms = ['refactor']

cc_actions = cc_adaptive_terms + cc_corrective_terms + cc_perfective_terms + cc_perfective_terms

cc_etc = ['breaking\s+change(!)?:']


def cc_title(astions):

    return '^(' + "|".join(astions) +")(\(.*\))?(!)?:"


# Adaptive
def build_cc_adaptive_regex():

    return cc_title(cc_adaptive_terms)


def is_cc_adaptive(commit_text):

    return len(re.findall(build_cc_adaptive_regex(), commit_text)) > 0


# Corrective
def build_cc_corrective_regex():

    return cc_title(cc_corrective_terms)


def is_cc_corrective(commit_text):

    return len(re.findall(build_cc_corrective_regex(), commit_text)) > 0

# Refactor
def build_cc_refactor_regex():

    return cc_title(cc_refactor_terms)


def is_cc_refactor(commit_text):

    return len(re.findall(build_cc_refactor_regex(), commit_text)) > 0

# Just Perfective
def build_cc_just_perfective_regex():

    return cc_title(cc_perfective_terms)


def is_cc_just_perfective(commit_text):

    return len(re.findall(build_cc_just_perfective_regex(), commit_text)) > 0

# Perfective
def build_cc_perfective_regex():

    return "(" + "|".join([build_cc_refactor_regex()
                              , build_cc_just_perfective_regex()]) + ")"


def is_cc_perfective(commit_text):

    return len(re.findall(build_cc_perfective_regex(), commit_text)) > 0

# CC message
def build_cc_message_regex():

    return "(" + "|".join([cc_title(cc_actions)] + cc_etc) + ")"



def is_cc_message(commit_text):

    return len(re.findall(build_cc_message_regex(), commit_text)) > 0




def print_cc_functions_for_bq(commit: str = 'XXX'):

    concepts = {'cc_adaptive' : build_cc_adaptive_regex
                , 'cc_corrective' : build_cc_corrective_regex
                , 'cc_refactor' : build_cc_refactor_regex
                , 'cc_just_perfective' : build_cc_just_perfective_regex
                , 'cc_perfective' : build_cc_perfective_regex
                , 'cc_message' : build_cc_message_regex
                }

    for i in concepts.keys():
        print()
        print_func = lambda : print_logic_to_bq(regex_func=concepts[i]
                                                , concept=i)
        generate_bq_function('{schema}.bq_{concept}'.format(schema=SCHEMA_NAME
                                                            , concept=i)
                             , print_func
                             , commit=commit)
        print()



## Corrective

### Keywords

In [ ]:
core_bug_terms = [
             'bug(s|z)?',
             'bug(?:-|\s)?fix(es)?',
             'defect(?:s)?',
             'error(?:s)?',
             'failur(?:ing|e|es|ed)',
             'fault(s)?',
             'fix(ed|es|ing)?',
             'fixing(?:s)?',
             'incorrect(ly)?',
             'mistake(s|n|d|nly)?',
             'problem(?:s)?',
             ]
# Positive
bug_terms = ['actual.*expected',
             '((assignment|assign|=) in if|== instead of =)',
             'expected.*actual'
             '(choose|take|set|use)\\s*(the|a)?\\s*correct', # correct as adjective
             "(not|isn't|doesn't)\\s+work(s|ing)?", # TODO - check with negation
             "doesn't recognize", # TODO Extend
             'double(?:-| )allocat(?:e|ion|ions)',
             'double(?:-| )free(?:s)?',
             "caused a regression", # TODO Extend
             'bad initialization(?:s)?',
             'buffer overflow(?:s)?',
             'fixme(?:s)?',
             'fixes(?:-| )?commit(?::| )?',
             '(break|breaks|broke|broked|breaking|broken)[\s\S]{0,20}(code|system|function|method)',
             '(this|that|it)\s(break|breaks|broke|broked|breaking|broken)',
             'break strict(?:-|\s)aliasing rule(s)?',
             'crash(?:ing|s|ed)?',
             'correct(?:ing|s|ed)?\\s*(a|the|some|few|this)', # make sure that correct serves as a verb
             'correct(ed|ion|ly|s)?',
             '(dangling|hanging) pointer(?:s)?',
             'deadlock(?:s)?',
             '(divid(e|es|ed|ing)|division) by (zero|0)',
             'double(?:-| )free',
             'fail(?:ing|s|ed)?',
             'faulty initialization(?:s)?',
             'fix(?:-| )?in(?:s)?',
             'fix(?:-| )?up(?:s)?',
             'flaw(?:s|ed)?',
             '(float|integer) (under|over)(?:-| )?flow',
             'hot(?:-| )?fix(?:ed|es|ing)?',
             #'hang',
             'heap overflow(?:s)?',
             '(?:im|im-)?proper'
             'memory(?:-| )?leak(?:s)?',
             'missing\s(default value|initialization|switch case)(?:s)?',
             'is\smissing',
             'add(?:ing|s|ed)?\smiss(?:ing|es|ed)?',
             #'must not',
             'npe(?:s)?'
             'null pointer(?:s)?',
             'off(?:-| )by(?:-| )(one|1)',
             'out of bound(?:s)?',
             'over(?:-| )?run(?:s)?',
             'patch(?:ed|ing)',
             #'prevent(?:ing|s|ed)?', # should be tuned
             'race condition(?:s)?',
             'data race(?:s)?',
             'repair(?:ing|s|ed)?',
             'resource leak(?:s)?',
             # TODO - check generalization to leaks works in the other direction to expected (reduces FP, increases FN)
             'leak(?:s)?',
             'revert(?:ing|s|ed)?',
             'segmentation (fault|violation)(?:s)?',
             'resolv(?:ing|e|es|ed)',
             #'solv(?:ing|e|es|ed)',
             'workaround(?:s)?',
             'wrong(nly)?',
             '(type(s)? mis(?:-| )?match|(not|non|none) matching type(s)?)',
             'trouble(?:s)?',
             '(un(?:-| )?|not )initialized variable(s)?',
             'unintended',
             'not intended',
             'unintentionally',
             'not intentionally',
             # 'unexpected.*occurred', # very rare, 90% are bugs anyway
             'vulnerabilit(?:y|ies)'
             ] + core_bug_terms

# Valid_fix_objects
valid_fix_object = prefective_entities + ['#',
                    '(camel|snake|kebab|flat|lower|upper)\\s*case',
                    "code review('s|s)?",
                    'comment(?:s)?',
                    'cosmetic(?:s)?',
                    'cr(s)?(?:-)?',
                    'documentation(?:s)?',
                    #'exception(?: |-)?handling',
                    #'format(s|ing)? fix(ed|es|ing)?',
                    'format(?:ing)?',
                    'help',
                    'remark(s)?',
                    'space(s)?',
                    'style|styling',
                    'typo(s)?',
                    'typing(?: |-)?(error|mistake)(s)?',
                    'warning(s)?',
                    'white(?: |-)?space(s)?']

valid_terms = [
    'break\sout',
    'error(?: |-)?check(ing)?',
    'error(?: |-)?handling',
    'error message(s)?',
    'error report(s|ing)?',
    'fixed(?: |-)?point',
    'fix(?:ed) ticket(?:s)?',
    #'format(ing)?',
    '(?:fix(?:ed|es)?|bug)(?: )?(?: |-|=|:)(?: )?[a-z]{0,3}(?:-)?\d+' + term_seperator,
    '(if|would)[\s\S]{0,40}go wrong',
    'line(?:s)? break(?:s)?',
    'typo(s)?\sfix(es)?',
    'fix(ed|es|ing)?' + build_sepereted_term(software_entities) + 'name(s)?',
    build_sepereted_term(static_analyzers) + 'fix(es|ed)?',
    'fix(es|ed)?' + build_sepereted_term(static_analyzers) ,
    '^### Bug Fix', # tends to be a title, later stating if the commit is a bug fix
    'edit the jira link to the correct issue', # Another occurring title
    'page(?:s)? break(?:s)?',


] + code_review_fixes



fixing_verbs = ['correct(?:ing|s|ed)'
                    , 'fix(ed|s|es|ing)?'
                    , 'repair(?:ing|s|ed)?'
                    ,  'revert(?:ing|s|ed)?'
                    , 'resolv(?:ing|e|es|ed)'
                    , 'revok(?:ing|e|es|ed)'
                    , 'und(?:oing|id)'
                ]
MERGE_PREFIX = '(merge (branch|pull request).{0,25}|merge (branch|pull request).{0,25}(from|into).{0,25})'
END_OF_LINE = r'(\r\n|\r|\n|$)'
corrective_header_entities = fixing_verbs + [
    'miss(?:ing|es|ed)?', 'should', 'must', '(have|has) to', 'avoid', 'prevent', 'break(s|ed|ing)?', 'broken'
    , 'remov(?:ing|e|es|ed) change(?:s)?', 'unable', 'proper(?:ly)?'
    , MERGE_PREFIX + "(%s)" % "|".join(core_bug_terms) + '.{0,250}' + END_OF_LINE
    #, "(does not|doesn't) need" , "cannot", "can not"
 ] #+ [ "do not" ,"don't", "dont"]

Regex Functions

In [ ]:
def build_sepereted_term(term_list : List, just_before =False):
    if just_before:
        sep = "%s(%s)" % (term_seperator, "|".join(term_list))
    else:
        sep = "%s(%s)%s" % (term_seperator, "|".join(term_list), term_seperator)
    return sep

In [ ]:
def build_valid_find_regex():
    fix_re = "(" + "|".join(fixing_verbs + [MERGE_PREFIX]) + ")"
    prefix = term_seperator + fix_re + '[\s\S]{1,40}' + "(" + "|".join(valid_fix_object) + ")" + term_seperator

    suffix = term_seperator + "(" + "|".join \
        (valid_fix_object) + ")" + term_seperator + '[\s\S]{0,40}' + term_seperator + fix_re + term_seperator

    # TODO - check seperation
    #sepertion = '(?:%s|%s[\s\S]{0,40}%s)' % (term_seperator, term_seperator, term_seperator)
    #suffix = "(" + "|".join \
    #    (valid_fix_object) + ")" + sepertion + fix_re + term_seperator

    #other_valid_re = "(%s)" % "|".join(valid_terms)
    other_valid_re = build_sepereted_term(valid_terms)
    return "((%s)|(%s)|(%s))" % (prefix, suffix, other_valid_re)

In [ ]:
def build_bug_fix_regex(use_conventional_commits=True):
    header_regex =  '(?:^|^[\s\S]{0,25}%s)(?:%s)%s' % (term_seperator
                                                       , "|".join(corrective_header_entities)
                                                       , term_seperator)
   # strict_header = "^(?:%s)%s"  % ( "|".join([ "do not" ,"don't"])
   #                                                    , term_seperator)

    bug_fix_re = build_sepereted_term(bug_terms)


    if use_conventional_commits:
        agg_re = "((%s)|(%s)|(%s))" % (bug_fix_re, header_regex, build_cc_corrective_regex())
    else:
        agg_re = "((%s)|(%s))" % (bug_fix_re, header_regex)

    return agg_re

In [ ]:
def build_negeted_bug_fix_regex():
    bug_fix_re = build_bug_fix_regex(use_conventional_commits=False)
    negation_re = build_sepereted_term(negation_terms)


    return "%s[\s\S]{0,20}%s" % (negation_re, bug_fix_re)

In [ ]:
def build_core_bug_regex():

    return '(%s)' % build_sepereted_term(core_bug_terms)


## Classification Functions

In [ ]:
def is_core_bug(commit_text):
    text = commit_text.lower()

    cnt = len(re.findall(build_core_bug_regex(), text))

    return cnt > 0

In [ ]:
def is_fix(commit_text):

    text = commit_text.lower()
    #text = normalize(text)

    fix_num = len(re.findall(build_bug_fix_regex(), text))
    valid_num = len(re.findall(build_valid_find_regex(), text))
    negated_num = len(re.findall(build_negeted_bug_fix_regex(), text))
    # TODO  consider modals
    #negated_num = len(re.findall(build_non_positive_linguistic(build_bug_fix_regex()), text))
    return (fix_num - valid_num - negated_num) > 0
    #return (fix_num ) > 0 # max recall with current predictor

In [ ]:
corrections = 0
for m in test['message']:
    print(m)
    print('CORRECTION: ' + str(is_fix(m)))
    print(' ')
    if is_fix(m) ==  True:
        corrections += 1
        
print(corrections)

In [ ]:
corrections = 0
for m in test['message']:
    print(m)
    print('CORRECTION: ' + str(is_core_bug(m)))
    print(' ')
    if is_core_bug(m) ==  True:
        corrections += 1
        
print(corrections)

## Security

### Keywords

In [ ]:
positive_terms = [
 'advisory',
 'attack(?:s)?',
 'auth',
 'authenticat(e|ion)',
 'brute force', # consider
 'bug bount(y|ies)',
 'bypass(?:es|ed|ing)?', # consider
 'certificate(?:s)?',
 #'constant time', # too general
 'crack',
 'credential(s)?',
 'cross(?: |-)origin',
 'cross(?: |-)site',
 'cve(-d+)?(-d+)?',
 'clickjack',
 'cyber',
 'denial of service',
 '(de)?serializ', # consider
 'directory traversal',
 'dos', # consider
 'exploit',
 #'expos(e|ing)',
 # 'hack', # A bit general, consider
 'hijack',
 'harden',
 #'infinite loop', # consider
 'injection',
 '(in)?secur(e|ity)',
 'lockout',
 'malicious',
 'malware(?:s)?', #plural of malware is malware yet not all are aware
 'nvd' # NVD
 'open redirect',
 'osvdb', # OSVDB
 'overflow', # consider
 'password(?:s)?',
 'permission(?:s)?',
 'poison(?:s|es|ed|ing)?',
 'port scan',
 'privilege(?:s)?',
 # 'proof of concept', # consider
 'rce', # remote code execution
 'redos' # ReDoS
 'remote code execution',
 'return oriented programming',
 '(?:safe|safety|unsafe|safer)',
 'security',
 'session fixation',
 'spoof(?:s|es|ed|ing)?',
 'threat(?:s|ed|ing)?',
 'timing', # consider
 'token(?:s)?',
 #'traversal',
 'unauthori[z|s]ed',
 'vulnerabilit(?:y|ies)',
 'x(?: |-)frame(?: |-)option(?:s)?',
 'xss',
 'xsrf', # XSRF
 'xxe' # XXE
    ]

excluded_terms = ['_____PLACEHOLDER_____'
                  ]

### Regex Functions

In [ ]:
def build_positive_regex():

    return build_sepereted_term(positive_terms)



def build_excluded_regex():

    return build_sepereted_term(excluded_terms)

def build_not_positive_regex():

    return build_non_positive_linguistic(build_positive_regex())


### Classification Functions

In [ ]:
def is_security(commit_text):

    return (len(re.findall(build_positive_regex(), commit_text))
            - len(re.findall(build_excluded_regex(), commit_text))
            - len(re.findall(build_not_positive_regex(), commit_text)))  > 0

In [ ]:
corrections = 0
for m in test['message']:
    print(m)
    print('CORRECTION: ' + str(is_security(m)))
    print(' ')
    if is_security(m) ==  True:
        corrections += 1
        
print(corrections)

## Adaptive

### Keywords

In [ ]:

core_adaptive_terms = [
    'add(?:s|ed|ing)?',
    'creat(?:e|es|ing)',
    'disabl(?:e|es|ed|ing)',
    'implement(?:ed|s|ing)?',
    'import(?:s|ed|ing)?',
    'introduc(?:e|es|ed|ing)',
    'port(?:s|ed|ing)?',
    'provid(?:e|es|ed|ing)',
    'updat(?:e|es|ed|ing)',
    'upgrad(?:e|es|ed|ing)'

]

adaptive_context = [
 '(?:un)?hid(?:e|es|den)',
 'allow(?:s|ed|ing)?',
 'buil(?:t|ds|ing)',
 'calibirat(?:e|es|ed|ing)',
 'configure',
 'deferr(?:ed|s|ing)?',
 'enhanc(?:e|es|ed|ing)',
 'extend(?:s|ed|ing)?',
 'form(?:ed|s|ing)?',
 'report(?:s|ed|ing)?',
 'support(s|ed|ing)?',

# , 'mov(e|es|ed|ing)'
# , 'print(s|ed|ing)?'

] + core_adaptive_terms



adaptive_entities = ['ability', 'configuration', 'conversion', 'debug', 'new', 'possibility', 'support'
    , 'test(s)?', 'tweak(s)?', 'mode', 'option']


adaptive_header_action = "|".join([
    'upgrad(?:e|es|ed|ing)',
    'configur(?:e|es|ed|ing)',
    'chang(?:e|es|ed|ing)',
    '(?:keep|change)\s+(?:the\s+)?default',
    'new',
    # '(?:make(?:s)?|made|making)',
    'merg(?:e|es|ed|ing)',
    'clear(?:s|ed|ing)?',
    #'comment(?:s|ed|ing)?\sout'
    'creat(?:e|es|ed|ing)',
    'cast(?:s|et|ing)?' + NEAR_ENOUGH + '\sas',
    # 'convert(?:s|ed|ing)?',
    # 'check(?:s|ed|ing)?',
    'add(?:s|ed|ing)?',
    # 'buil(?:d|t|ds|ing)',
    'Initial revision',
    '(?:im)?port(?:s|ed|ing)?',
    '(?:un)?hid(?:e|es|den)',
    'updat(?:e|es|ed|ing)',
    'upload(?:s|ed|ing)?',
    'disabl(?:e|es|ed|ing)',
    'delet(?:e|es|ed|ing)',
    'enabl(?:e|es|ed|ing)',
    'quirk(?:s|ed|ing)?',
    'skip(?:s|ed|ing)?',
    'switch(?:s|ed|ing)?',
    'allow(?:s|ed|ing)?',
    'provid(e|es|ed|ing)',

    ###
    # , 'build'
    # , 'mark(?:s|ed|ing)?'
    # , 'us(?:e|es|ed|ing)'
    # , '(?:make|made|making)'
    # , 'creat(?:e|es|ed|ing)'
    # , 'handl(?:e|es|ed|ing)'
    'remov(?:e|es|ed|ing)',
    'refresh(?:s|ed|ing)?',
    #'re(-)?enabl(?:e|es|ed|ing)',

] +no_message
)

adaptive_actions = [  # 'revert(?:s|ed|ing)?',
    #'merg(?:e|es|ed|ing)[\s\S]{1,5}(pull request|pr|branch)',
    'add(?:s|ed|ing)?[\s\S]{1,50}(?:version|v\d|ver\d)',
    #'(cr(s)?(-)?|code\sreview)\sfix(?:s|ed|ing)?',
    '(^|\s)implement(?:ed|s|ing)?\s',
    '(?:make(?:s)?|made|making)[\s\S]{1,50}consitent',
    'updat(?:e|es|ed|ing)[\s\S]{1,25}to[\s\S]{1,25}\d+.\d',
    'updat(?:e|es|ed|ing)\s+(to\s+)?\d+\.\d',
    '(?:add(s|ed|ing)?|delet(?:e|es|ed|ing)|updat(?:e|es|ed|ing))\s+' + file_scheme,
    # '(add(s|ed|ing)?|delet(e|es|ed|ing)|updat(e|es|ed|ing))\s+([A-Z0-9_]*)', # TODO - run without lower
    '(^|^[\s\S]{0,25}%s)(%s)%s' % (term_seperator, adaptive_header_action, term_seperator),
    # '^(?:version|v\d+\.\d|ver\d+\.\d)',
    '^\[(?:IMP|imp)\]',  # TODO - take care of upper/lower case
    'support(?:s|ed|ing)?\sfor\s',
    'show(?:es|ed|ing)?[\s\S]instead',
    'scal(?:e|es|ed|ing)?\s(up|down)'

                   ] + code_review_fixes

In [ ]:
def build_adaptive_action_regex():

    return "(%s)" % ("|".join(
    adaptive_actions))




def build_adaptive_regex(use_conventional_commits=True):

    adaptive_context_re = build_sepereted_term(adaptive_context, just_before=True)


    base_re = "((%s)\s[\s\S]{0,50}(%s)%s)" % (adaptive_context_re
                            ,  "|".join(adaptive_entities + software_terms)
                            , term_seperator)

    if use_conventional_commits:
        agg_re = "((%s)|(%s))" % (base_re
                                  , build_cc_adaptive_regex())
    else:
        agg_re = base_re

    return agg_re



def build_non_adaptive_context():

    non_adaptive_header_action = "|".join([
                                'transla(?:tion|et|eted|ets|ting)'
                                ,  'readme(?:.md)?'
                              ])

    non_adaptive_header ='^[\s\S]{0,50}(%s)' % non_adaptive_header_action

    entities = documentation_entities + ['bug',
                'helper',
                'miss(?:ing|ed)',
                'to(?: |-)?do(?:s)?',
                'warning(?:s)?'
                ]

    adaptive_actions = ['remov(?:e|es|ed|ing)']
    non_adaptive_entities = documentation_entities + software_terms + unnedded_terms + [file_scheme]


    return '(%s)' % "|".join(['(?:%s)\s[\s\S]{0,50}(?:%s)' % (build_sepereted_term(adaptive_context, just_before=True)
                                                            , "|".join(entities))
                     , non_adaptive_header
                     , '(?:%s)\s[\s\S]{0,50}(?:%s)' % (build_sepereted_term(adaptive_actions, just_before=True)
                                                            , "|".join(non_adaptive_entities))
                     ])


def build_non_adaptive_linguistic():

    return build_non_positive_linguistic(build_adaptive_regex(use_conventional_commits=False))

def build_core_adaptive_regex():

    return '(%s)' % build_sepereted_term(core_adaptive_terms)


#### Classification Functions

In [ ]:
def is_adaptive(text):

    return (match(text, build_adaptive_regex())
            + match(text, build_adaptive_action_regex())
            - match(text, build_non_adaptive_context())
            - match(text, build_non_adaptive_linguistic()))


In [ ]:
adaptive = 0
for m in test['message']:
    print(m)
    print('CORRECTION: ' + str(is_adaptive(m)))
    print(' ')
    if is_adaptive(m) ==  True:
        adaptive += 1
        
print(adaptive)

## Refactor

### Keywords

In [ ]:
# https://arxiv.org/pdf/2002.11049.pdf
refactor_entities = software_terms + ['(helper|utility|auxiliary) function(?:s)?']


# Well, we need them...
unnedded_terms = ['unnecessary', 'unneeded', 'unused', '(?:not|never|no longer) used'
    #, 'old'
    , 'no longer needed', 'redundant', 'useless', 'duplicate(?:d)?', 'deprecated', 'obsolete(?:d)?', 'commented']

core_refactor_terms = [
    'clean(?:ing)?(?:-| )?up(?:s)?',
    'clean(?:ing|s|ed)?',
    'combin(?:e|es|ed|ing)',
    'compos(?:e|es|ed|ing)',
    'de(?:-| )?compos(?:e|es|ed|ing)',
    'deprecat(?:e|es|ed|ing)',
    'encapsulat(?:e|es|ed|ing)',
    'polish(?:ed|es|ing)?',
    're(?:-| )?factor(?:ed|s|ing|ings)?', # TODO - should be here - check why slightly decreases performance
    're(?:-|)?organiz(?:e|es|ed|ing)',
    're(?:-|)?structur(?:e|es|ed|ing)',
    'rebuil(?:d|ds|ding|t)',
    'tid(?:y|ying|ied)'
]


modification_activity = [
'(?:get|got|getting) rid',
 '(?:make|makes|made|making)',
 'convert(?:ed|s|ing)?',
 'dead',
 'drop(?:ed|s|ing)?',
 'duplicat(?:e|es|ed|ing)',
 'extract(?:ed|s|ing)?',
 'hide(?:e|es|ed|ing)',
 'improv(?:e|es|ed|ing)',    # Goals modification only?
 'increas(?:e|es|ed|ing)',
 'mov(?:e|es|ed|ing)',
 'parameteriz(?:e|es|ed|ing)',
 'redundant',
 'replac(?:e|es|ed|ing)',
 'separat(?:e|e s|ed|ing)',
 'short(:?en|er|ing|s)?',
 'split(?:s|ing)?',
 'subsitut(?:e|es|ed|ing)',
 'substitut(?:e|es|ed|ing)',
 'un(?:-| )?hid(?:e|es|ed|ing)'

    #'chang(?:e|esed|ing)'
    #, 'creat(?:e|es|ed|ing)'
    #, 'delet(?:e|es|ed|ing)'
    #, 'instead'
    #, 'kill(?:ed|s|ing)?'
    # , 'provid(?:e|es|ed|ing)'
    #, 'introduc(?:e|es|ed|ing)'
] + core_refactor_terms + unnedded_terms

feedbak_terms = [ 'py(?:-| )?lint', 'lint', 'review comments(?:s)?', 'code review', 'cr', 'pep8'
                  ]
feedback_action = ['fix(?:ed|s|es|ing)?', 'fix(?:-| )?up(?:s)?', 'resolv(?:e|ed|es|ing)', 'correct(?:ed|s|es|ing)?']

perfective_header_action = [
    #'polish(?:ed|es|ing)?'
    #, 'clean(?:ing|s|ed)?(?:-| )?up(?:s)?'
     'clean(?:ing|s|ed)?(?:-| )?up(?:s)?'
    , 'cleaner'
    , 'deprecat(?:e|es|ed|ing)'
    , 'extract(?:ed|s|ing)?',
    're(?:-|)?organiz(?:e|es|ed|ing)', 're(?:-|)?structur(?:e|es|ed|ing)', 'tid(?:y|ying|ied) up'
    , 'improv(?:e|ed|es|ing|ement|ements)' , 're(?:-|)?organiz(?:e|es|ed|ing)', 're(?:-|)?structur(?:e|es|ed|ing)'
    , '(helper|utility|auxiliary) function(?:s)?'
    , '(?:move|moved|moves|moving) to'
    , 'separat(?:e|es|ed|ing)'
    , 'split(?:s|ing)?', '->'
    , build_sepereted_term(static_analyzers) + 'fix(es|ed)?'
    , 'fix(es|ed)?' + build_sepereted_term(static_analyzers)

    #, '(private|public|protected|static)'
]

# TODO - rewrited, move into/out???, deduplicate, remove legacy, redo, PR, feedback

# TODO - clean , style, prettier, "->", refine, "Removed commented code", "More startup improvements.", recode
# ""Remove another old function", "improved redis error message", utility functions, never used
# Checkstyle


# TODO - perfective, not refactor - ident, spacing, tabs, "tabs -> spaces", cosmetic, ""*** empty log message ***"
# examples ""DOC: remove mention of TimeSeries in docs"

# TODO - add "resolving review comments"
# TODO - lint, pylint
refactor_context = [ 'clean(ing)?(-| )?up(s)?'
    ,'call(?:s|ed|ing)?[\s\S]{1,50}instead'
    , 'collaps(?:e|es|ed|ing)', 'consolidat(e|es|ed|ing)'
    , 'decompos(?:e|es|ed|ing)'
    , 'drop(?:ed|s|ing)?( back)', 'encapsulat(e|es|ed|ing)'
    , 'gereneliz(?:e|es|ed|ing)'
                    # , 'inline'
                    # , 'no longer needed', 'not used', 'obsolete(d)?'
    , 'optimiz(?:e|es|ed|ing|ation|ations)'
    , 'pull(?:ed|s|ing)? (up|down)', 're(?:-)?(?:write|wrote)', 're(?:-| )?factor(?:ed|s|ing|ings)?'
    , 're(-)?implement(ed|s|ing)?'
    , 'renam(?:e|es|ed|ing|ings)', 'better nam(?:e|es|ing)','re(?:-)?organiz(e|es|ed|ing)', 're(?:-)?organization'
    , 're(?:-)?work(ed|s|ing|ings)?'
    , 'reorg' , 'simplif(y|es|ied|ying|ication)', 'suppress(es|ed|ing)? warning(?:s)?'
    , 'unif(?:y|ies|ied|ing)', 'uninline'
    , 'beef(?:ed|s|ing)? up', 'refactor(?:ing)?(?:s)?', 'code improvement(?:s)?'
    #, '(?:^|^[\s\S]{0,25}%s)(?:%s)%s[\s\S]{0,25}$' % (term_seperator, "|".join(perfective_header_action), term_seperator)
    , 'revis(?:e|es|ed|ing)'
    , 're(?:-)?construct(?:s|ed|ing)?'
    , 're(?:-)?(?:write|write|wrote|writing)'
    , 're(?:-)?cod(?:e|ed|es|ing)'
    , 'factor(?:ed|s|ing)? out'
    , 're(?:-| )?packag(?:e|es|ed|ing)'
    #, 'code review'
    #, 'collapse'
    #, "(?:(?:%s)(?:%s|%s[\s\S]{0,50}%s)(?:%s)%s)" % (build_sepereted_term(feedback_action
    #                                                                                      , just_before=True)
    #                                                                 , term_seperator
    #                                                                 , term_seperator
    #                                                                 , term_seperator
    #                                                                 , "|".join(feedbak_terms)
    #                                                                 , term_seperator)
                    # ,'us(e|es|ed|ing)[\s\S]{1,50}(instead)'
                    # , '(instead)[\s\S]{1,50}us(e|es|ed|ing)'
                    ]
# https://refactoring.guru/refactoring/techniques

# TODO - change [\s\S] with . ?
removal = [ 'add(?:s|ed|ing)?[\s\S]{1,50}helper(?:s)?'
    ,  'us(?:e|es|ed|ing)[\s\S]{1,50}instead'
    #,  'us(?:e|es|ed|ing)[\s\S]{1,25}\(\)[\s\S]{1,25}instead'
    ,  'split(?:s|ing)?[\s\S]{1,50}into'
    ,  'break(?:s|ing)?[\s\S]{1,50}into'
    ,  'separat(?:e|e s|ed|ing)[\s\S]{1,50}into'
    #,  'replac(?:e|es|ed|ing)?[\s\S]{1,50}with'
    ,  'replac(?:e|es|ed|ing)?[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    , 'remov(?:e|es|ed|ing)[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    #, '(?:this|that|is)[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    ,  'kill(?:s|ed|ing)?[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    ,  'drop(?:s|ed|ing)?[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    ,  'mov(?:e|es|ed|ing)?[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
            ]
adaptive_context = [
    '(?:un)?hid(?:e|es|den)', 'add(?:s|ed|ing)?', 'allow(?:s|ed|ing)?'
    , 'buil(?:t|ds|ing)', 'calibirat(?:e|es|ed|ing)'
    , 'configure'
    , 'creat(?:e|es|ing)' #   O created
    , 'deferr(?:ed|s|ing)?'
    , 'disabl(?:e|es|ed|ing)'
    , 'enhanc(?:e|es|ed|ing)', 'extend(?:s|ed|ing)?', 'form(?:ed|s|ing)?'
    , 'implement(?:ed|s|ing)?', 'import(?:s|ed|ing)?', 'introduc(?:e|es|ed|ing)'
    , 'port(?:s|ed|ing)?'
    , 'provid(?:e|es|ed|ing)'
    , 'report(?:s|ed|ing)?'
    , 'support(s|ed|ing)?'
    , 'updat(?:e|es|ed|ing)'
    , 'upgrad(?:e|es|ed|ing)'

    # , 'mov(e|es|ed|ing)'
    # , 'print(s|ed|ing)?'


]

### Regex Functions

In [ ]:
def build_core_refactor_regex():

    return '(%s)' % build_sepereted_term(core_refactor_terms)

def build_refactor_regex(use_conventional_commits=True):
    header_regex =  '(?:^|^[\s\S]{0,25}%s)(?:%s)%s' % (term_seperator
                                                       , "|".join(perfective_header_action)
                                                       , term_seperator)

    activity_regerx = "(?:(?:%s)(?:%s|%s[\s\S]{0,50}%s)(?:%s)%s)" % (build_sepereted_term(modification_activity
                                                                                          , just_before=True)
                                                                     , term_seperator
                                                                     , term_seperator
                                                                     , term_seperator
                                                                     , "|".join(refactor_entities)
                                                                     , term_seperator)
    if use_conventional_commits:
        agg_re = "(%s)|(%s)|(%s)|(%s)" % (build_sepereted_term(refactor_context)
                          , activity_regerx
                          , header_regex
                          , build_cc_refactor_regex())
    else:
        agg_re = "(%s)|(%s)|(%s)" % (build_sepereted_term(refactor_context)
                          , activity_regerx
                          , header_regex)
    return agg_re



def build_refactor_goals_regex():
    goals_regerx = "(?:(?:%s)(?:%s|%s[\s\S]{0,50}%s)(?:%s)%s)" % (build_sepereted_term(software_goals_modification
                                                                                       , just_before=True)
                                                                  , term_seperator
                                                                  , term_seperator
                                                                  , term_seperator
                                                                  , "|".join(software_goals)
                                                                  , term_seperator)
    return goals_regerx


def build_non_code_perfective_regex():

    non_perfective_entities = ['warning(?:s)?'
                               , 'format(?:ing)?'
                               , 'indentation(?:s)?'
                              ]
    # TODO - applied to perfective entities too here, which is a bug.
    modification_action = ['clean(?:-| )?up(?:s)?']
    non_perfective_context = [
                            'fix(?:es|ed|ing)?'
                            ,'(?:get|got|getting) rid'
                            , 'support(?:s|ed|ing)?'
                            ]
    modifiers = modification_activity + non_perfective_context
    activity_regerx = "((?:%s)(?:\s|%s[\s\S]{0,50}%s)(?:%s))" % (build_sepereted_term(modifiers, just_before=True)
                                                                                , term_seperator
                                                                                , term_seperator
                                                                                , "|".join(prefective_entities
                                                                                           + non_perfective_entities))
    doc_header_regex =  '(?:^|^[\s\S]{0,25}%s)(?:%s)[\s\S]{0,25}(?:%s)' % (term_seperator
                                                       , "|".join(perfective_header_action)
                                                       , build_sepereted_term(documentation_entities))


    no_prefective_action = "|".join([
        'convert(?:ed|s|ing)?(?:%s|%s[\s\S]{0,50}%s)support(?:s|ed|ing)?' % (
            term_seperator,term_seperator, term_seperator)
        , '(?:make|made|making|makes)(?:%s|%s[\s\S]{0,50}%s)work' % (term_seperator, term_seperator, term_seperator)
        , '(?:make|made|making|makes)(?:%s|%s[\s\S]{0,50}%s)sense' % (term_seperator, term_seperator, term_seperator)
        , 'improv(?:e|es|ed|ing) handling'
        , 'need(?:s|ing)?\srefactor(?:ing)?'
        , '(?:%s)(?:%s|%s[\s\S]{0,50}%s)(?:%s)' %(build_sepereted_term(non_perfective_entities,just_before=True)
                                                   ,term_seperator
                                                   , term_seperator
                                                   , term_seperator
                                                   , "|".join(modification_action)
                                                   )
        , doc_header_regex

    ])
    non_perfective_context = '(?:%s|%s)' % (no_prefective_action
                                         , activity_regerx)

    return non_perfective_context


def build_documentation_entities_context(positive_re):

    return '(?:%s)' % "|".join([
        # TODO - take care of documentation entities spereatly
         '(?:%s)[\s\S]{0,10}(?:%s)' % (build_sepereted_term(documentation_entities, just_before=True)
                                        ,positive_re)
    ])

def build_perfective_regex():
    non_code = build_sepereted_term (prefective_entities)

    perfective = "(%s)" %  non_code

    return perfective

Classification Functions

In [ ]:
def built_is_refactor(commit_text):
    removal_re = build_sepereted_term(removal)

    return (match(commit_text, build_refactor_regex())
            + match(commit_text, removal_re)
            + match(commit_text, build_refactor_goals_regex())
            - match(commit_text, build_non_code_perfective_regex())
            - match(commit_text
                    , build_documentation_entities_context(build_refactor_regex(use_conventional_commits=False)))
            - match(commit_text
                    , build_non_positive_linguistic(build_refactor_regex(use_conventional_commits=False)))
            - match(commit_text, build_non_positive_linguistic(build_sepereted_term(removal)))
            - match(commit_text, build_non_positive_linguistic(build_refactor_goals_regex()))
            ) > 0


In [ ]:
refactors = 0
for m in test['message']:
    print(m)
    print('CORRECTION: ' + str(built_is_refactor(m)))
    print(' ')
    if built_is_refactor(m) ==  True:
        refactors += 1
        
print(refactors)

# Analysis

### Timeframe  2000 to 2020

In [ ]:
df = df.loc[df['date'] > '2000-01-01'].loc[df['date'] < '2021-01-01']

In [ ]:
df['date']

In [ ]:
df.loc[df['date'] > '2000-01-01'].loc[df['date'] < '2021-01-01']

In [ ]:
df

In [ ]:
df.shape

In [ ]:
%%time
#df['is_adpative'] = df[0:100]['message'].apply(lambda x: is_adaptive(str(x)))
df['is_core_bug'] = df['message'].apply(lambda x: is_core_bug(str(x)))

In [ ]:
%%time
#df['is_fix'] = df['message'].apply(lambda x: is_fix(str(x)))

In [ ]:
with open('data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

## Load classified data

In [ ]:
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)


In [ ]:
df = data

In [ ]:
df

In [ ]:
commits_sum = df.pivot_table(index='repo', columns='dday', values='message', aggfunc='count')

In [ ]:
commits_sum = commits_sum.fillna(0).cumsum(axis=1)

In [ ]:
commits_sum

In [ ]:
commits_sum.loc['rstudio/rstudio'].dropna().plot.area()

In [ ]:
commits_sum.loc['tensorflow/tensorflow'].plot()

In [ ]:
commits_sum.loc['apache/httpd'].plot()

In [ ]:
commits_sum.where(commits_sum > 0).loc['kubernetes/kubernetes'].dropna().plot.area()

In [ ]:
repo_lang = pd.read_json('https://api.github.com/repos/apache/httpd/languages', orient='index')

repo_lang.plot.pie(figsize=(10,10), autopct='%1.1f%%',
        shadow=True, startangle=90, subplots=True, fontsize=16)

In [ ]:
sns.lineplot(commits_sum.where(commits_sum > 0).loc['kubernetes/kubernetes'].dropna())

In [ ]:
commits_sum

In [ ]:
df.head()

In [ ]:
summary = df.pivot_table(index='repo', values='message', aggfunc='count', margins=True)
summary

In [ ]:
users = df.groupby('repo')['user'].nunique()
summary = summary.join(users)

In [ ]:
summary

In [ ]:
summary['user'][-1] = summary['user'].dropna().sum()

In [ ]:
summary

In [ ]:
summary['message %'] = (summary['message']/summary.loc['All']['message']).apply(lambda x: "{:.1%}".format(x))
summary['user %'] = (summary['user']/summary.loc['All']['user']).apply(lambda x: "{:.1%}".format(x))
summary

In [ ]:
df['dday'][0:100]

In [ ]:
df['week'] = df['dday'].apply(lambda x: x.strftime('%W'))

In [ ]:
week = df.pivot_table(index='repo',columns=['week','year'],values='message', aggfunc='count')

In [ ]:
week.mean(axis=1)

In [ ]:
#bugs = df.pivot_table(index='repo', values=['is_core_bug','is_fix'], aggfunc='sum')
bugs = df.pivot_table(index='repo', values=['is_core_bug'], aggfunc='sum')

In [ ]:
bugs = bugs.join(summary)

In [ ]:
bugs['is_not_core_bug'] = bugs['message'] - bugs['is_core_bug']

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
fig.autofmt_xdate(rotation=90)
bugs['bug %'] = bugs['is_core_bug']/bugs['message']
bugs['other %'] = bugs['is_not_core_bug']/bugs['message']
bugs = bugs.sort_values('bug %')
ax.bar(bugs.index, bugs['bug %'], label='bug', color='#f00000')
ax.bar(bugs.index, bugs['other %'], label='other', bottom=bugs['bug %'])
ax.legend()

In [ ]:
commit_days = df.pivot_table(index='repo',columns=['dday'],values='message', aggfunc='count')

In [ ]:
days = df.pivot_table(index='repo',columns=['dday'],values='is_core_bug', aggfunc='sum')

In [ ]:
fig.autofmt_xdate(rotation=90)
pd.plotting.boxplot(days.T, figsize=(15,5))

In [ ]:
sm_stats.descriptivestats.describe(commit_days.T)

In [ ]:
commit_days.describe()

In [ ]:
for repo in days.index:
    fig, ax = plt.subplots()
    ax.plot(days.loc[repo].dropna())
    ax.set_title(repo)
    #days.loc[repo].dropna().plot()

In [ ]:
days.loc['tidyverse/ggplot2'].reset_index().reset_index()

In [ ]:
scaler = preprocessing.StandardScaler()
day_norm = scaler.fit_transform(days.T).T
day_norm

In [ ]:
days.mean().hist()

In [ ]:
"""
for repo in days.index:
    fig, ax = plt.subplots()
    ax.hist(day_norm.loc[repo])
    #sns.lmplot(x='index',y=repo,data=days.loc[repo].reset_index().reset_index(), markers='.', palette="Set1")
    #ax.plot(days.loc[repo].dropna())
    ax.set_title(repo)
    #days.loc[repo].dropna().plot()
    
#sns.regplot(x='index', y='tidyverse/ggplot2' ,data=days.loc['tidyverse/ggplot2'].dropna().reset_index().reset_index())
"""

## KDE

In [ ]:
from sklearn.neighbors import KernelDensity

In [ ]:
df.head()

In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.2)

In [ ]:
kde.fit(np.array(df.loc[df['repo'] == 'pandas-dev/pandas']['is_core_bug']).reshape(1,-1))

### Bug Corrections Daily Proportions

In [ ]:
commit_months = commit_days.T.asfreq('M')

In [ ]:
bug_months = days.T.asfreq('M')

In [ ]:
bug_month_ratio = bug_months/commit_months

In [ ]:
initial_date = {}
for repo in bug_ratio.columns:
    initial_date[repo] = bug_ratio[repo].dropna().head(1).index
initial_date

In [ ]:
bug_month_ratio['matplotlib'] = bug_ratio.loc[bug_ratio.index > '2010-01-01']['matplotlib/matplotlib'].fillna(0)

In [ ]:
bug_month_ratio['ggplot2'] = bug_ratio.loc[bug_ratio.index > '2010-01-01']['tidyverse/ggplot2'].fillna(0)

## Tests

### Binomial Test

In [ ]:
#sm_stats.proportion.binom_test(count, nobs, prop=0.5, alternative='two-sided')
bin_test = {}
for repo in df['repo'].unique():
    count = df.loc[df['repo'] == repo]['is_core_bug'].sum()
    nobs = df.loc[df['repo'] == repo]['is_core_bug'].count()
    bin_test[repo] = sm_stats.proportion.binom_test(count, nobs, prop=0.5, alternative='smaller')


In [ ]:
pd.Series(bin_test).sort_index()

In [ ]:
bug_ratio.columns

In [ ]:
pairs = [['tensorflow/tensorflow','pytorch/pytorch'],['tidyverse/ggplot2','matplotlib/matplotlib'],['denoland/deno','nodejs/node'],
         ['facebook/react','angular/angular'],['apache/httpd', 'nginx/nginx'], ['rstudio/rstudio', 'jupyterlab/jupyterlab']]

In [ ]:
bin_test2 = {}
for repo in pairs:
    print(repo[0])
    print(repo[1])
    count1 = df.loc[df['repo'] == repo[0]]['is_core_bug'].sum()
    nobs1 = df.loc[df['repo'] == repo[0]]['is_core_bug'].count()
    count2 = df.loc[df['repo'] == repo[1]]['is_core_bug'].sum()
    nobs2 = df.loc[df['repo'] == repo[1]]['is_core_bug'].count()
    #print(count1, nobs1, count2, nobs2 )
    #print(count1/nobs1, count2/nobs2 )
    bin_test2[repo[0]] = sm_stats.proportion.test_proportions_2indep(count1, nobs1, count2, nobs2, value=None, method=None, compare='diff', alternative='smaller', correction=True, return_results=False)    

In [ ]:
bin_test2

In [ ]:
pd.DataFrame(bin_test2).T.rename({0:'stats',1:'p-value'})

In [ ]:
[ bin_test2[k][1] < 0.05 for k in bin_test2]

In [ ]:
pairs_initial_date = []
for pair in pairs:
    if initial_date[pair[0]] > initial_date[pair[1]]:
        pairs_initial_date.append(0)
    else:
        pairs_initial_date.append(1)
pairs_initial_date        

### Normality

In [ ]:
shapiro_test = stats.shapiro(bug_ratio['matplotlib/matplotlib'].dropna())
shapiro_test

In [ ]:
stats.kstest(bug_ratio['matplotlib/matplotlib'].dropna(), 'norm')

In [ ]:
bug_month_ratio['matplotlib'].plot()

In [ ]:
bug_month_ratio['ggplot2'].plot()

### Two sample tests

In [ ]:
stats.ttest_ind(bug_month_ratio['matplotlib'], bug_month_ratio['ggplot2'])

In [ ]:
stats.ks_2samp(bug_ratio['matplotlib/matplotlib'].dropna(), bug_ratio['tidyverse/ggplot2'].dropna())

In [ ]:
stats.wilcoxon(bug_month_ratio['matplotlib'], bug_month_ratio['ggplot2'])

### Time Series

In [ ]:
kpss = {}
for repo in days.index:
    kpss[repo] = tsa.stattools.kpss(days.loc[repo].dropna(), nlags=90)

In [ ]:
tsa.stattools.kpss(days.loc['pytorch/pytorch'], nlags=120)

In [ ]:
pd.DataFrame(kpss).T